In [1]:
from submodules import Objects, Robots, Tasks, Users
from submodules.Scenes import Scene
from submodules.Actions import Actions
from submodules.Gestures import Gestures
from submodules.Users import Users
from model_noK import *
import numpy as np
from numpy import array as a
import os

In [2]:
def create_observation_compact(s,TaTo):
    target_object_id = s.O.index(TaTo[1])
    target_action_id = Actions.A.index(TaTo[0])
    target_object_type_id = Object.all_types.index(getattr(s, TaTo[1]).type)
    
    gesture_probs_intent = CM[target_action_id,:]
    performed_gesture = np.random.choice(np.arange(lenG),p = gesture_probs_intent,size=1)
    gesture_vec = np.zeros(lenG)
    gesture_vec[performed_gesture] = 1
    gesture_vec = abs(gesture_vec + np.random.normal(loc = 0, scale = 0.2,size =(1,lenG)))[0]
    gesture_vec = gesture_vec/np.sum(gesture_vec)
    return gesture_vec 

In [20]:
def show_each_action_occurance(dataset):
    sums = np.zeros(lenA)
    for sample in dataset:
        a = A.index(sample[1][0])
        sums[a]+=1
    print(f"Action occurances: {sums}")

In [16]:
def save_with_config(dataset, name='saves'):
    config = {
        'A': A,
        'G': G,
        'Otypes': Otypes,
        'lenA': lenA,
        'lenG': lenG,
        'lenOtypes': lenOtypes,
        'CM': CM,
    }
    np.save(f"{os.path.expanduser('~')}/Documents/{name}", np.asanyarray({'dataset': dataset, 'config': config}, dtype=object))

# Generating data

Ex1: Generate random action draws with execution. Draws are from one scene session.

In [5]:
Actions.A = ['move_up', 'open', 'put', 'pour', 'close', 'pick_up', 'put_on_target']

Function `regulize_repeating_pattern` checks if the current action draw is not the same as the last two draws. 

In [6]:
def regulize_repeating_pattern(TaTos, TaTo):
    if len(TaTos) < 2: return True
    
    if TaTo == TaTos[-1] or TaTo == TaTos[-2]:
        return False
    return True

## Ex.1: Generate random draws from all actions

Generate random valid action draws from the input created scene.

Each action has `common_sense_probability` threshold `p>0.9` which rejects the action that has probability under this value.
If any action is not found over `common_sense_probability` threshold. The next draws are chosen based on if the action is feasible/doable `p>0.01`.

In [7]:
draws = 20
common_sense_probability_threshold = 0.9
feasibility_probability_threshold = 0.01

In [8]:
# Generate random scene
s = Scene(init='drawer,cup,cup,object')

TaTos = []
# Generate random action
for i in range(draws):
    n=0
    while True:
        # random draw from all actions
        TaTo = (Actions.get_random_action(), s.get_random_object())
        # regulizer
        m=0
        while not regulize_repeating_pattern(TaTos, TaTo):
            TaTo = (Actions.get_random_action(), s.get_random_object())
            m+=1
            if m>100: raise Exception("Cannot find solution")
        
        success = Actions.do(s, TaTo, ignore_location=True, p=common_sense_probability_threshold)
        if n>100:
            success = Actions.do(s, TaTo, ignore_location=True, p=feasibility_probability_threshold)
        if n>1000: raise Exception("Cannot find solution 2")
        if success:
            TaTos.append(TaTo)
            break
        n+=1

In [9]:
TaTos

[('put', 'object'),
 ('close', 'drawer'),
 ('pick_up', 'cup1'),
 ('put_on_target', 'object'),
 ('open', 'drawer'),
 ('close', 'drawer'),
 ('pick_up', 'cup1'),
 ('put_on_target', 'object'),
 ('open', 'drawer'),
 ('pick_up', 'cup1'),
 ('put_on_target', 'drawer'),
 ('close', 'drawer'),
 ('pick_up', 'object'),
 ('put_on_target', 'drawer'),
 ('open', 'drawer'),
 ('pick_up', 'cup1'),
 ('put', 'drawer'),
 ('close', 'drawer'),
 ('pick_up', 'object'),
 ('put_on_target', 'object')]

In [10]:
s.info

Scene info. shape: [4, 4, 4]
drawer,	drawer,	[1 3 0], closed, cont: ['cup', 'cup1'],	|| [drawer] >>
cup,	cup,	[1 0 0], empty,	|| [cup] >>
cup1,	cup,	[2 0 0], empty,	|| [cup1] >>
object,	object,	[3 1 0],	|| [object] >>
Robot: [1 0 3], opened, rotation: 0, attached: -


TaTos variable contains possible actions which can be executed from newly created scene.
Note that objects might need to be on the same position as the previous scene.

In [13]:
s = Scene(init='drawer,cup,cup,object')
for TaTo in TaTos:
    Actions.do(s, TaTo)

In [14]:
s.info

Scene info. shape: [4, 4, 4]
drawer,	drawer,	[2 0 0], opened, cont: ['cup'],	|| [drawer] >>
cup,	cup,	[3 2 0], full,	|| [cup] >>
cup1,	cup,	[1 1 0], empty,	|| [cup1] >>
object,	object,	[1 3 0],	|| [object] >>
Robot: [3 3 3], opened, rotation: 1, attached: -


## Ex. 2: Generate random draws from possible actions
- Same as Ex.1, except draws are made only from possible action -> better distribution output

In [15]:
# Generate random scene
s = Scene(init='drawer,cup,cup')

TaTos = []
# Generate random action
for i in range(draws):
    n=0
    while True:
        # Random draw from possible actions
        TaTo = Actions.get_random_possible_action(s, ignore_location=True, p=common_sense_probability_threshold)
        flag_feasible_option = False
        while not regulize_repeating_pattern(TaTos, TaTo):
            flag_feasible_option = False
            TaTo = Actions.get_random_possible_action(s, ignore_location=True, p=common_sense_probability_threshold)
            if TaTo is False:
                flag_feasible_option = True
                TaTo = Actions.get_random_possible_action(s, ignore_location=True, p=feasibility_probability_threshold)
                
        # Action execution
        if flag_feasible_option:
            success = Actions.do(s, TaTo, ignore_location=True, p=feasibility_probability_threshold)
        else:
            success = Actions.do(s, TaTo, ignore_location=True, p=common_sense_probability_threshold)
        if success:
            TaTos.append(TaTo)
            break


In [16]:
TaTos

[('close', 'drawer'),
 ('put_on_target', 'cup'),
 ('open', 'drawer'),
 ('pick_up', 'cup1'),
 ('close', 'drawer'),
 ('put_on_target', 'cup1'),
 ('pick_up', 'cup'),
 ('put_on_target', 'drawer'),
 ('pick_up', 'cup1'),
 ('put_on_target', 'cup'),
 ('open', 'drawer'),
 ('close', 'drawer'),
 ('pick_up', 'cup1'),
 ('put_on_target', 'drawer'),
 ('open', 'drawer'),
 ('close', 'drawer'),
 ('pick_up', 'cup1'),
 ('put_on_target', 'cup1'),
 ('pick_up', 'cup'),
 ('open', 'drawer')]

# Dataset generation for classification task

### Ex. 4: dataset_01 generator

- 6 Actions + 6 Gestures
- move_up is not feasible currently (0 samples) and pour has small probability to be used (\~50 samples), close and pick_up are most probable (\~300 samples) 
- 1000 draws
- CM is diagonal 2D
- types, scene and other features are not used
- scene: 2 drawers, 3 cups, initialized randomly

In [17]:
from submodules import Robots, Tasks, Users
from submodules.Objects import Object
from submodules.Scenes import Scene
from submodules.Actions import Actions
from submodules.Gestures import Gestures
from submodules.Users import Users
from model_noK import *
import numpy as np
from numpy import array as a

In [22]:
A = Actions.A = ['move_up', 'open', 'put', 'pour', 'close', 'pick_up']
G = Actions.G = ['swipe_up', 'open', 'swipe_back', 'hold', 'rotate', 'swipe_forward']
Otypes = Object.all_types
lenA = len(A)
lenG = len(G)
lenOtypes = len(Otypes)

In [23]:
CM = np.diag(np.ones(lenA))

In [24]:
draws = 1000
common_sense_probability_threshold = 0.9
feasibility_probability_threshold = 0.01

In [25]:
dataset = []
# Generate random action
for i in range(draws):
    s = Scene(init='drawer,drawer,cup,cup,cup', random=True)
    # Random draw from possible actions
    TaTo = Actions.get_random_possible_action(s, ignore_location=True, p=common_sense_probability_threshold)
    
    gobs = create_observation_compact(s,TaTo)
    
    dataset.append((s.to_dict(), TaTo, gobs))

In [26]:
save_with_config(dataset, name='dataset01')

View how different actions were drafted

In [27]:
show_each_action_occurance(dataset)

Action occurances: [  0. 137. 167.  47. 355. 294.]


### Ex. 5: dataset02 generator

- 6 Actions + 6 Gestures
- 1000 draws
- CM is 2D, with picked values:
```
          | gestures:
actions:| swipe_up, open, swipe_back, hold, rotate, swipe_forward
move_up | [[0.7,    0.3,  0.0,        0,    0,       0  ],         
open    | [0.2,     0.4,  0.4,        0,    0,       0  ],        
put     | [0.0,     0.4,  0.6,        0,    0,       0  ],        
pour    | [0.0,     0.0,  0.0,        0.2,  0.8,     0.0],        
close   | [0.0,     0.0,  0.0,        0.2,  0.0,     0.8],        
pick_up | [0.7,     0.0,  0.0,        0.3,  0.0,     0.0]]        
```
- types, scene and other features are not used
- scene: 2 drawers, 3 cups, initialized randomly

In [28]:
from submodules import Robots, Tasks, Users
from submodules.Objects import Object
from submodules.Scenes import Scene
from submodules.Actions import Actions
from submodules.Gestures import Gestures
from submodules.Users import Users
from model_noK import *
import numpy as np
from numpy import array as a
import os

In [29]:
A = Actions.A
G = Gestures.G
Otypes = Object.all_types
lenA = len(A)
lenG = len(G)
lenOtypes = len(Otypes)

In [30]:
CM = np.zeros((lenA,lenG))
CM[:,:] = np.array([[0.7, 0.3, 0.0, 0, 0, 0],
                    [0.2, 0.4, 0.4, 0, 0, 0],
                    [0.0, 0.4, 0.6, 0, 0, 0],
                    [0.0, 0.0, 0.0, 0.2, 0.8, 0.0],
                    [0.0, 0.0, 0.0, 0.2, 0.0, 0.8],
                    [0.7, 0.0, 0.0, 0.3, 0.0, 0.0]])

In [31]:
draws = 1000
common_sense_probability_threshold = 0.9
feasibility_probability_threshold = 0.01

In [32]:
dataset = []
# Generate random action
for i in range(draws):
    s = Scene(init='drawer,drawer,cup,cup,cup', random=True)
    # Random draw from possible actions
    TaTo = Actions.get_random_possible_action(s, ignore_location=True, p=common_sense_probability_threshold)
    
    gobs = create_observation_compact(s,TaTo)
    
    dataset.append((s.to_dict(), TaTo, gobs))

In [33]:
save_with_config(dataset, name='dataset02')

In [34]:
show_each_action_occurance(dataset)

Action occurances: [  0. 136. 185.  44. 337. 298.]


### Ex. 6: dataset03 generator

- Put all-in
- 7 Actions + 7 Gestures
- 1000 draws (10000 draws as `dataset03_10000d`)
- CM is 4D `[2,2,7,7]`
- types, scene and other features are not used
- scene: 2 drawers, 3 cups, initialized randomly

In [35]:
from submodules import Robots, Tasks, Users
from submodules.Objects import Object
from submodules.Scenes import Scene
from submodules.Actions import Actions
from submodules.Gestures import Gestures
from submodules.Users import Users
from model_noK import *
import numpy as np
from numpy import array as a

In [36]:
A = Actions.A = ['move_up', 'open', 'put', 'pour', 'close', 'pick_up', 'put_on_target']
G = Gestures.G = ['swipe_up', 'open', 'swipe_back', 'hold', 'rotate', 'swipe_forward', 'swipe_left']
Otypes = Object.all_types
lenA = len(A)
lenG = len(G)
lenOtypes = len(Otypes)

In [37]:
CM = np.zeros((2,2, lenA,lenG)) # users x object types x actions x gestures
# user 0, objtyp 0
CM[0,0,:,:] = [[0.6, 0, 0.4, 0, 0, 0, 0], #two gestures for moving up the cup
               [0  , 0, 0  , 0, 0, 0, 0], #target action for cup cannot be open or close (zero probs.)
               [0  , 1, 0  , 0, 0, 0, 0], #same gesture for move front as for open
               [0  , 0, 0  , 1, 0, 0, 0],
               [0  , 0, 0  , 0, 0, 0, 0],
               [0  , 0, 0  , 0, 0, 1, 0],
               [0  , 0, 0  , 0, 0, 0, 1]]
CM[1,0,:,:] = [[0.4, 0, 0.6, 0, 0,  0  , 0],
               [0  , 0, 0  , 0, 0,  0  , 0],
               [0  , 1, 0  , 0, 0,  0  , 0],
               [0  , 0, 0  , 1, 0,  0  , 0],
               [0  , 0, 0  , 0, 0,  0  , 0], #target action for cup cannot be open or close
               [0  , 0, 0  , 0, 0.2,0.8, 0],
               [0  , 0, 0  , 0, 0.2,0.8, 0]]
CM[0, 1, :, :] = [[0, 0, 0, 0, 0, 0, 0],
                  [0, 1, 0, 0, 0, 0, 0],
                  [0, 0, 1, 0, 0, 0, 0],
                  [0, 0, 0, 1, 0, 0, 0],
                  [0, 0, 0, 0, 1, 0, 0],
                  [0, 0, 0, 0, 0, 1, 0],
                  [0, 0, 0, 0, 0, 0, 1]]
CM[1, 1, :, :] = [[0.4, 0, 0.6, 0, 0  , 0  , 0],
                  [0  , 1, 0  , 0, 0  , 0  , 0],
                  [0  , 1, 0  , 0, 0  , 0  , 0],
                  [0  , 0, 0  , 1, 0  , 0  , 0],
                  [0  , 0, 0  , 0, 0.8, 0.2, 0], #two gestures for closing drawer
                  [0  , 0, 0  , 0, 0  , 0  , 0],
                  [0  , 0, 0  , 0, 0  , 0  , 1]]

In [38]:
draws = 1000
common_sense_probability_threshold = 0.9
feasibility_probability_threshold = 0.01

In [39]:
def create_observation_compact(s,TaTo):
    target_object_id = s.O.index(TaTo[1])
    target_action_id = Actions.A.index(TaTo[0])
    target_object_type_id = Object.all_types.index(getattr(s, TaTo[1]).type)
    
    gesture_probs_intent = CM[0,target_object_type_id,target_action_id,:]
    performed_gesture = np.random.choice(np.arange(lenG),p = gesture_probs_intent,size=1)
    gesture_vec = np.zeros(lenG)
    gesture_vec[performed_gesture] = 1
    gesture_vec = abs(gesture_vec + np.random.normal(loc = 0, scale = 0.2,size =(1,lenG)))[0]
    gesture_vec = gesture_vec/np.sum(gesture_vec)
    return gesture_vec 

In [40]:
dataset = []
# Generate random action
for i in range(draws):
    s = Scene(init='drawer,drawer,cup,cup,cup', random=True)
    # Random draw from possible actions
    TaTo = Actions.get_random_possible_action(s, ignore_location=True, p=common_sense_probability_threshold)
    
    gobs = create_observation_compact(s,TaTo)
    
    dataset.append((s.to_dict(), TaTo, gobs))

In [41]:
save_with_config(dataset, name='dataset03')

In [42]:
show_each_action_occurance(dataset)

Action occurances: [   0.  794.  846.  268. 2367. 2936. 2789.]


### Scene all absolute informations in: dataset04_all_scene_info

In [1]:
from submodules import Robots, Tasks, Users
from submodules.Objects import Object
from submodules.Scenes import Scene
from submodules.Actions import Actions
from submodules.Gestures import Gestures
from submodules.Users import Users
from model_noK import *
import numpy as np
from numpy import array as a

In [2]:
A = Actions.A = ['move_up', 'open', 'put', 'pour', 'close', 'pick_up', 'put_on_target']
G = Gestures.G = ['swipe_up', 'open', 'swipe_back', 'hold', 'rotate', 'swipe_forward', 'swipe_left']
Otypes = Object.all_types
lenA = len(A)
lenG = len(G)
lenOtypes = len(Otypes)

In [3]:
CM = np.zeros((2,2, lenA,lenG)) # users x object types x actions x gestures
# user 0, objtyp 0
CM[0,0,:,:] = [[0.6, 0, 0.4, 0, 0, 0, 0], #two gestures for moving up the cup
               [0  , 0, 0  , 0, 0, 0, 0], #target action for cup cannot be open or close (zero probs.)
               [0  , 1, 0  , 0, 0, 0, 0], #same gesture for move front as for open
               [0  , 0, 0  , 1, 0, 0, 0],
               [0  , 0, 0  , 0, 0, 0, 0],
               [0  , 0, 0  , 0, 0, 1, 0],
               [0  , 0, 0  , 0, 0, 0, 1]]
CM[1,0,:,:] = [[0.4, 0, 0.6, 0, 0,  0  , 0],
               [0  , 0, 0  , 0, 0,  0  , 0],
               [0  , 1, 0  , 0, 0,  0  , 0],
               [0  , 0, 0  , 1, 0,  0  , 0],
               [0  , 0, 0  , 0, 0,  0  , 0], #target action for cup cannot be open or close
               [0  , 0, 0  , 0, 0.2,0.8, 0],
               [0  , 0, 0  , 0, 0.2,0.8, 0]]
CM[0, 1, :, :] = [[0, 0, 0, 0, 0, 0, 0],
                  [0, 1, 0, 0, 0, 0, 0],
                  [0, 0, 1, 0, 0, 0, 0],
                  [0, 0, 0, 1, 0, 0, 0],
                  [0, 0, 0, 0, 1, 0, 0],
                  [0, 0, 0, 0, 0, 1, 0],
                  [0, 0, 0, 0, 0, 0, 1]]
CM[1, 1, :, :] = [[0.4, 0, 0.6, 0, 0  , 0  , 0],
                  [0  , 1, 0  , 0, 0  , 0  , 0],
                  [0  , 1, 0  , 0, 0  , 0  , 0],
                  [0  , 0, 0  , 1, 0  , 0  , 0],
                  [0  , 0, 0  , 0, 0.8, 0.2, 0], #two gestures for closing drawer
                  [0  , 0, 0  , 0, 0  , 0  , 0],
                  [0  , 0, 0  , 0, 0  , 0  , 1]]

In [4]:
draws = 1000
common_sense_probability_threshold = 0.9
feasibility_probability_threshold = 0.01

In [ ]:
def create_observation_compact(s,TaTo):
    target_object_id = s.O.index(TaTo[1])
    target_action_id = Actions.A.index(TaTo[0])
    target_object_type_id = Object.all_types.index(getattr(s, TaTo[1]).type)
    
    gesture_probs_intent = CM[0,target_object_type_id,target_action_id,:]
    performed_gesture = np.random.choice(np.arange(lenG),p = gesture_probs_intent,size=1)
    gesture_vec = np.zeros(lenG)
    gesture_vec[performed_gesture] = 1
    gesture_vec = abs(gesture_vec + np.random.normal(loc = 0, scale = 0.2,size =(1,lenG)))[0]
    gesture_vec = gesture_vec/np.sum(gesture_vec)
    return gesture_vec 

In [ ]:
dataset = []
# Generate random action
for i in range(draws):
    s = Scene(init='cup,drawer,object', random=True)
    # Random draw from possible actions
    TaTo = Actions.get_random_possible_action(s, ignore_location=True, p=common_sense_probability_threshold)
    
    gobs = create_observation_compact(s,TaTo)
    
    dataset.append((s.to_dict(), TaTo, gobs))

In [ ]:
save_with_config(dataset, name='dataset04_all_scene_info')

In [ ]:
show_each_action_occurance(dataset)

# Task 10.9. Generate three datasets
### Ds. 1: dataset2_1 generator

- Direct mapping + scene + target object
- 11 Actions + 9 Gestures
- 10000 draws
- CM is:
    - `dataset2_1` 2D `[11,9]`
    - `dataset2_2` 3D `[3,11,9]`
    - `dataset2_3` 4D `[2,3,11,9]`
- types, scene and other features are not used
- scene: n objects - initialized randomly
- target object generation type: **random**

In [1]:
from submodules import Robots, Tasks, Users
from submodules.Objects import Object
from submodules.Scenes import Scene
from submodules.Actions import Actions
from submodules.Gestures import Gestures
from submodules.Users import Users
from model_noK import *
import numpy as np
from numpy import array as a
import os

In [2]:
A = Actions.A = ['move_up', 'move_left', 'move_down', 'move_right', 
                 'put', 'put_on', 'pour', 'pick_up', 'place', 'open', 'close']

G = Gestures.G = ['swipe_up', 'swipe_left', 'swipe_down', 'swipe_right', 
                  'five', 'grab', 'thumbsup', 'rotate', 'point']
Otypes = Object.all_types
lenA = len(A)
lenG = len(G)
lenOtypes = len(Otypes)
object_types_order_list = ['cup', 'drawer', 'object', 'cup', 'drawer', 'object', 'cup']

In [22]:
CM1 = np.zeros((lenA,lenG)) # actions x gestures
                    # up,left,dwn,rght,fiv,grab,thum,rota,poin
CM1[:,:] = np.array([[.7,  .0, .0,  .0, .3, .0,   .0,  .0,  .0], # move_up
                     [.0,  .7, .0,  .0, .1, .0,   .1,  .1,  .0], # move_left
                     [.0,  .0, .7,  .0, .2, .0,   .0,  .1,  .0], # move_down
                     [.0,  .0, .0,  .7, .3, .0,   .0,  .0,  .0], # move_right
                     [.0,  .0, .4,  .0, .2, .0,   .0,  .0,  .4], # put
                     [.0,  .0,  1,  .0, .0, .0,   .0,  .0,  .0], # put_on
                     [.0,  .0, .0,  .0, .0, .0,   .1,  .9,  .0], # pour
                     [.2,  .0, .0,  .0, .0, .7,   .1,  .0,  .0], # pick_up
                     [.0,  .0, .0,  .0, .0, .0,   .3,  .0,  .7], # place
                     [.5,  .0, .0,  .0, .0, .5,   .0,  .0,  .0], # open
                     [.0,  .0, .5,  .0, .0, .5,   .0,  .0,  .0]])# close

In [23]:
CM2 = np.zeros((3, lenA,lenG)) # object types x actions x gestures
# objtyp 0: cup
                        # up,left,dwn,rght,fiv,grab,thum,rota,poin
CM2[0,:,:] = np.array([[.7,  .0, .0,  .0, .3, .0,   .0,  .0,  .0], # move_up
                       [.0,  .8, .0,  .0, .0, .0,   .1,  .1,  .0], # move_left
                       [.0,  .0, .7,  .0, .2, .0,   .0,  .1,  .0], # move_down
                       [.0,  .0, .0,  .7, .3, .0,   .0,  .0,  .0], # move_right
                       [.0,  .0, .0,  .0, .0, .0,   .0,  .0,  .0], # put
                       [.0,  .0, .0,  .0, .0, .0,   .0,  .0,  .0], # put_on
                       [.0,  .0, .0,  .0, .0, .0,   .1,  .9,  .0], # pour
                       [.2,  .0, .0,  .0, .0, .7,   .1,  .0,  .0], # pick_up
                       [.0,  .0, .0,  .0, .0, .0,    1,  .0,  .0], # place
                       [.0,  .0, .0,  .0, .0, .0,   .0,  .0,  .0], # open
                       [.0,  .0, .0,  .0, .0, .0,   .0,  .0,  .0]])# close
# user 0, objtyp 1: drawer
                      # up,left,dwn,rght,fiv,grab,thum,rota,poin
CM2[1,:,:] = np.array([[.7,  .0, .0,  .0, .3, .0,   .0,  .0,  .0], # move_up
                       [.0,  .7, .0,  .0, .1, .0,   .1,  .1,  .0], # move_left
                       [.0,  .0, .7,  .0, .2, .0,   .0,  .1,  .0], # move_down
                       [.0,  .0, .0,  .7, .3, .0,   .0,  .0,  .0], # move_right
                       [.0,  .0, .4,  .0, .2, .0,   .0,  .0,  .4], # put
                       [.0,  .0, .5,  .0, .0, .0,   .0,  .0,  .5], # put_on
                       [.0,  .0, .0,  .0, .0, .0,   .0,  .0,  .0], # pour
                       [.0,  .0, .0,  .0, .0, .0,   .0,  .0,  .0], # pick_up
                       [.0,  .0, .0,  .0, .0, .0,   .3,  .0,  .7], # place
                       [.4,  .0, .0,  .0, .0, .4,   .2,  .0,  .0], # open
                       [.0,  .0, .5,  .0, .0, .5,   .0,  .0,  .0]])# close
# objtyp 2: object = box
                      # up,left,dwn,rght,fiv,grab,thum,rota,poin
CM2[2,:,:] = np.array([[.7,  .0, .0,  .0, .3, .0,   .0,  .0,  .0], # move_up
                       [.0,  .7, .0,  .0, .1, .0,   .1,  .1,  .0], # move_left
                       [.0,  .0, .7,  .0, .2, .0,   .0,  .1,  .0], # move_down
                       [.0,  .0, .0,  .7, .3, .0,   .0,  .0,  .0], # move_right
                       [.0,  .0, .4,  .0, .2, .0,   .0,  .0,  .4], # put
                       [.0,  .0, .5,  .0, .0, .0,   .0,  .0,  .5], # put_on
                       [.0,  .0, .0,  .0, .0, .0,   .0,  .0,  .0], # pour
                       [.2,  .0, .0,  .0, .0, .7,   .1,  .0,  .0], # pick_up
                       [.0,  .0, .0,  .0, .0, .0,   .3,  .0,  .7], # place
                       [.0,  .0, .0,  .0, .0, .0,   .0,  .0,  .0], # open
                       [.0,  .0, .0,  .0, .0, .0,   .0,  .0,  .0]])# close

In [35]:
CM3 = np.zeros((2,3, lenA,lenG)) # users x object types x actions x gestures
CM3[0,0,:,:] = np.array([[ 1,  .0, .0,  .0,  0, .0,   .0,  .0,  .0], # move_up
                         [.0,   1, .0,  .0, .0, .0,    0,   0,  .0], # move_left
                         [.0,  .0,  1,  .0,  0, .0,   .0,   0,  .0], # move_down
                         [.0,  .0, .0,   1,  0, .0,   .0,  .0,  .0], # move_right
                         [.0,  .0, .0,  .0, .0, .0,   .0,  .0,  .0], # put
                         [.0,  .0, .0,  .0, .0, .0,   .0,  .0,  .0], # put_on
                         [.0,  .0, .0,  .0, .0, .0,    0,   1,  .0], # pour
                         [.0,  .0, .0,  .0, .0,  1,    0,  .0,  .0], # pick_up
                         [.0,  .0, .0,  .0, .0, .0,    0,  .0,   1], # place
                         [.0,  .0, .0,  .0, .0, .0,   .0,  .0,  .0], # open
                         [.0,  .0, .0,  .0, .0, .0,   .0,  .0,  .0]])# close
# objtyp 1: drawer
                      # up,left,dwn,rght,fiv,grab,thum,rota,poin
CM3[0,1,:,:] = np.array([[ 1,  .0, .0,  .0,  0, .0,   .0,  .0,  .0], # move_up
                         [.0,   1, .0,  .0,  0, .0,    0,   0,  .0], # move_left
                         [.0,  .0,  1,  .0,  0, .0,   .0,   0,  .0], # move_down
                         [.0,  .0, .0,   1,  0, .0,   .0,  .0,  .0], # move_right
                         [.0,  .0,  0,  .0,  0, .0,   .0,  .0,   1], # put
                         [.0,  .0,  0,  .0, .0, .0,    1,  .0,   0], # put_on
                         [.0,  .0, .0,  .0, .0, .0,   .0,  .0,  .0], # pour
                         [.0,  .0, .0,  .0, .0, .0,   .0,  .0,  .0], # pick_up
                         [.0,  .0, .0,  .0, .0,  1,    0,  .0,   0], # place
                         [ 0,  .0, .0,  .0, .0,  0,    0,   1,  .0], # open
                         [.0,  .0,  0,  .0,  1,  0,   .0,  .0,  .0]])# close
# user 0, objtyp 2: object = box
                        # up,left,dwn,rght,fiv,grab,thum,rota,poin
CM3[0,2,:,:] = np.array([[ 1,  .0, .0,  .0,  0, .0,   .0,  .0,  .0], # move_up
                         [.0,   1, .0,  .0,  0, .0,    0,   0,  .0], # move_left
                         [.0,  .0,  1,  .0,  0, .0,   .0,   0,  .0], # move_down
                         [.0,  .0, .0,   1,  0, .0,   .0,  .0,  .0], # move_right
                         [.0,  .0,  0,  .0,  1, .0,   .0,  .0,   0], # put
                         [.0,  .0,  0,  .0, .0, .0,   .0,  .0,   1], # put_on
                         [.0,  .0, .0,  .0, .0, .0,   .0,  .0,  .0], # pour
                         [ 0,  .0, .0,  .0, .0,  1,    0,  .0,  .0], # pick_up
                         [.0,  .0, .0,  .0, .0, .0,    1,  .0,   0], # place
                         [.0,  .0, .0,  .0, .0, .0,   .0,  .0,  .0], # open
                         [.0,  .0, .0,  .0, .0, .0,   .0,  .0,  .0]])# close
# user 1, objtyp 0: cup
                        # up,left,dwn,rght,fiv,grab,thum,rota,poin
CM3[0,0,:,:] = np.array([[.7,  .0, .0,  .0, .3, .0,   .0,  .0,  .0], # move_up
                         [.0,  .8, .0,  .0, .0, .0,   .1,  .1,  .0], # move_left
                         [.0,  .0, .7,  .0, .2, .0,   .0,  .1,  .0], # move_down
                         [.0,  .0, .0,  .7, .3, .0,   .0,  .0,  .0], # move_right
                         [.0,  .0, .0,  .0, .0, .0,   .0,  .0,  .0], # put
                         [.0,  .0, .0,  .0, .0, .0,   .0,  .0,  .0], # put_on
                         [.0,  .0, .0,  .0, .0, .0,   .1,  .9,  .0], # pour
                         [.2,  .0, .0,  .0, .0, .7,   .1,  .0,  .0], # pick_up
                         [.0,  .0, .0,  .0, .0, .0,    1,  .0,  .0], # place
                         [.0,  .0, .0,  .0, .0, .0,   .0,  .0,  .0], # open
                         [.0,  .0, .0,  .0, .0, .0,   .0,  .0,  .0]])# close
# user 1, objtyp 1: drawer
                      # up,left,dwn,rght,fiv,grab,thum,rota,poin
CM3[0,1,:,:] = np.array([[.7,  .0, .0,  .0, .3, .0,   .0,  .0,  .0], # move_up
                         [.0,  .7, .0,  .0, .1, .0,   .1,  .1,  .0], # move_left
                         [.0,  .0, .7,  .0, .2, .0,   .0,  .1,  .0], # move_down
                         [.0,  .0, .0,  .7, .3, .0,   .0,  .0,  .0], # move_right
                         [.0,  .0, .4,  .0, .2, .0,   .0,  .0,  .4], # put
                         [.0,  .0, .5,  .0, .0, .0,   .0,  .0,  .5], # put_on
                         [.0,  .0, .0,  .0, .0, .0,   .0,  .0,  .0], # pour
                         [.0,  .0, .0,  .0, .0, .0,   .0,  .0,  .0], # pick_up
                         [.0,  .0, .0,  .0, .0, .0,   .3,  .0,  .7], # place
                         [.4,  .0, .0,  .0, .0, .4,   .2,  .0,  .0], # open
                         [.0,  .0, .5,  .0, .0, .5,   .0,  .0,  .0]])# close
# user 1, objtyp 2: object = box
                      # up,left,dwn,rght,fiv,grab,thum,rota,poin
CM3[0,2,:,:] = np.array([[.7,  .0, .0,  .0, .3, .0,   .0,  .0,  .0], # move_up
                         [.0,  .7, .0,  .0, .1, .0,   .1,  .1,  .0], # move_left
                         [.0,  .0, .7,  .0, .2, .0,   .0,  .1,  .0], # move_down
                         [.0,  .0, .0,  .7, .3, .0,   .0,  .0,  .0], # move_right
                         [.0,  .0, .4,  .0, .2, .0,   .0,  .0,  .4], # put
                         [.0,  .0, .5,  .0, .0, .0,   .0,  .0,  .5], # put_on
                         [.0,  .0, .0,  .0, .0, .0,   .0,  .0,  .0], # pour
                         [.2,  .0, .0,  .0, .0, .7,   .1,  .0,  .0], # pick_up
                         [.0,  .0, .0,  .0, .0, .0,   .3,  .0,  .7], # place
                         [.0,  .0, .0,  .0, .0, .0,   .0,  .0,  .0], # open
                         [.0,  .0, .0,  .0, .0, .0,   .0,  .0,  .0]])# close

In [36]:
draws = 10000
common_sense_probability_threshold = 0.9
feasibility_probability_threshold = 0.01

In [43]:
def create_observation(s,TaTo, CM_dim=3):
    target_object_id = s.O.index(TaTo[1])
    target_action_id = Actions.A.index(TaTo[0])
    target_object_type_id = Object.all_types.index(getattr(s, TaTo[1]).type)
    user_id = np.random.randint(2)
    
    if CM_dim == 2:
        gesture_probs_intent = CM1[target_action_id,:]
    elif CM_dim == 3:
        gesture_probs_intent = CM2[target_object_type_id,target_action_id,:]
    elif CM_dim == 4:
        gesture_probs_intent = CM3[user_id,target_object_type_id,target_action_id,:]
    if sum(gesture_probs_intent) == 0:
        return False, False
    performed_gesture = np.random.choice(np.arange(lenG),p = gesture_probs_intent,size=1)

    gesture_vec = np.zeros(lenG)
    gesture_vec[performed_gesture] = 1
    gesture_vec = abs(gesture_vec + np.random.normal(loc = 0, scale = 0.2,size =(1,lenG)))[0]
    gesture_vec = gesture_vec/np.sum(gesture_vec)
    
    focus_point = abs(np.random.normal(loc = s.objects[target_object_id].position, scale = 0.1,size =(1,3)))
    
    return gesture_vec, focus_point

In [49]:
dataset = []
# Generate random action
# for i in range(draws):
while len(dataset) < draws:

    n_objects = round(np.random.normal(3.5,1))
    init = ",".join(object_types_order_list[0:n_objects])
    
    s = Scene(init=init, random=True)
    # Random draw from possible actions
    TaTo = Actions.get_random_possible_action(s, ignore_location=True, p=common_sense_probability_threshold)
    if TaTo is False: continue
    
    gobs, oobs = create_observation(s,TaTo,CM_dim=4)
    if gobs is False:
        continue
    
    dataset.append((s.to_dict(), TaTo, gobs, oobs))
    if i % 1000 == 0: print(i)

In [50]:
len(dataset)

10000

In [51]:
CM = CM3
save_with_config(dataset, name='dataset2_3_10000d')

In [52]:
show_each_action_occurance(dataset)

Action occurances: [   0. 1850. 2566. 1854.  489.  257.   31. 1029. 1010.  248.  666.]


In [30]:
1

1

In [180]:
TaTo

False

D1 action occurances: [   0. 1838. 2564. 1848.  473.  276.   32. 1061. 1040.  214.  644.]

D2 action occurances: [   0. 1854. 2504. 1913.  507.  253.   23. 1047. 1009.  221.  653.]

D3 action occurances: [   0. 1850. 2566. 1854.  489.  257.   31. 1029. 1010.  248.  666.]